## 필요 라이브러리 import

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## 글번호 가져오기

In [4]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Whale/3.25.232.19 Safari/537.36"
}
gall_nums = []
for i in range(1, 12):
    response = requests.get(
        f"https://gall.dcinside.com/mgallery/board/lists/?id=school_police&page={i}",
        headers=headers
    )
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    items = soup.select(".gall_num")
    for item in items:
        number = item.text.strip()
        if number.isdigit():
            gall_nums.append(number)

In [6]:
print(gall_nums)

['862', '861', '860', '859', '858', '855', '854', '852', '849', '847', '837', '831', '829', '828', '826', '825', '824', '814', '811', '808', '805', '804', '802', '801', '800', '799', '798', '797', '796', '795', '793', '790', '789', '788', '787', '786', '784', '783', '782', '781', '780', '776', '773', '772', '771', '770', '769', '765', '763', '761', '760', '759', '758', '756', '754', '753', '752', '750', '749', '748', '747', '744', '743', '742', '741', '740', '739', '738', '729', '728', '725', '723', '721', '719', '718', '717', '716', '715', '714', '713', '712', '710', '708', '707', '705', '704', '703', '698', '697', '696', '692', '691', '688', '682', '681', '679', '678', '677', '676', '675', '674', '673', '672', '671', '669', '668', '666', '665', '664', '663', '661', '660', '658', '657', '656', '651', '650', '645', '644', '643', '641', '638', '637', '635', '634', '633', '631', '628', '627', '625', '624', '623', '621', '620', '618', '613', '610', '607', '603', '602', '601', '600', '598'

## 내용 크롤링

In [7]:
data = []
for i in gall_nums:
    response = requests.get(
        f"https://gall.dcinside.com/mgallery/board/view/?id=school_police&no={i}",
        headers=headers)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    subject = soup.select_one(".title_subject").text.strip()
    contents = soup.select(".write_div>div>div")
    for content in contents:
        for sentence in content.text.splitlines():
            sentence = sentence.strip()
            if len(sentence) < 4:
                continue
            data.append([i, subject, sentence])

df = pd.DataFrame(data, columns=["글번호", "제목", "내용"])
df["글번호"] = df["글번호"].astype(int)

df.to_excel("dcinside_응답하라학교폭력갤러리_줄바꿈단위.xlsx", index=False)

## DCinside Crawler

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

last_page = 27  # 마지막 페이지 번호
base_url = "https://gall.dcinside.com"
list_url = "/mini/board/lists/"
view_url = "/mini/board/view/"
gallery_id = "oursorrowfuleyes"
final_url = f"{base_url}{list_url}?id={gallery_id}"
filename = "dcinside_우울증대피소미니갤러리"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Whale/3.25.232.19 Safari/537.36"
}

# 글번호 수집
gall_nums = []
for i in range(1, last_page + 1):
    response = requests.get(
        f"{base_url}{list_url}?id={gallery_id}&page={i}",
        headers=headers, timeout=10
    )
    soup = BeautifulSoup(response.text, "html.parser")
    items = soup.select(".gall_num")
    for item in items:
        number = item.text.strip()
        if number.isdigit():
            gall_nums.append(number)

gall_nums = list(set(gall_nums))  # 중복 제거

# 글 내용 수집
data = []
for i in gall_nums:
    try:
        response = requests.get(
            f"{base_url}{view_url}?id={gallery_id}&no={i}",
            headers=headers, timeout=10
        )
        soup = BeautifulSoup(response.text, "html.parser")
        subject_tag = soup.select_one(".title_subject")
        subject = subject_tag.text.strip() if subject_tag else ""
        contents = soup.select(".write_div>div>div")
        for content in contents:
            for sentence in content.text.splitlines():
                sentence = sentence.strip()
                if len(sentence) < 4 or sentence.startswith(('http', 'www')):
                    continue
                data.append([i, subject, sentence])
    except Exception as e:
        print(f"Error processing post {i}: {e}")

df = pd.DataFrame(data, columns=["글번호", "제목", "내용"])
df["글번호"] = df["글번호"].astype(int)
df.to_excel(filename+"_crawlingData.xlsx", index=False)


In [12]:
print(f"{base_url}&page=1")

https://gall.dcinside.com/mgallery/board/lists/?id=fucking&page=1
